## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [1]:
# Import libraries and initialize file paths for each country
import os
import pandas as pd
import numpy as n
from typing import Generator

def get_file_paths(root: str) -> Generator[str, None, None]:
    """
    Returns the paths for all the files in the subtree of `root`
    """
    for (dir_path, _, file_names) in os.walk(root):
        yield from [os.path.join(dir_path, file_name) for file_name in file_names]

gn_file_paths = [*get_file_paths("./data/guinea_data")]
lr_file_paths = [*get_file_paths("./data/liberia_data")]
sl_file_paths = [*get_file_paths("./data/sl_data")]

In [208]:
def normalize_daily_data(raw_frame: pd.core.frame.DataFrame, variable_col):
    """
    Normalize the dataframe obtained by reading a a daily CSV to have
    one row per (region/date) pair and use columns for each variable.
    """
    # Normalize column names
    raw_frame.columns = map(str.lower, raw_frame.columns)
    
    DATE: str = "date"
    VARIABLE: str = variable_col
    REGION: str = "region"
    VALUE: str = "value"
    
    values_stack = []
    var_names = [] # Index of variables

    # Iterate over each variable (row)
    for index, row in raw_frame.iterrows():
        date = row[DATE]
        var_name = row[VARIABLE]
        if var_name in var_names:
            # Ignore variable: it was already defined
            continue
        var_names.append(var_name)
        
        # Iterate over each region (column)
        for col_name, value in row.iteritems():
            if col_name in {DATE, VARIABLE, "western area combined", "national"}:
                # Ignore: not a region
                continue
            values_stack.append({DATE: pd.to_datetime(date), REGION: col_name, VARIABLE: var_name, VALUE: value})
            
    stack_frame = pd.DataFrame(values_stack)
    pivoted = stack_frame.set_index([DATE, REGION, VARIABLE]).unstack(VARIABLE)
    # Drop to level ("value") of the column's multi index
    pivoted.columns = [col[1] for col in pivoted.columns] 
    return pivoted
    
def normalize_sl(raw_frame: pd.core.frame.DataFrame):
    """
    Normalize the dataframe obtained by reading a Sierra Leone CSV to have
    one row per (region/date) pair and use columns for each variable.
    """
    return normalize_daily_data(raw_frame, "variable")

sl = pd.concat([normalize_sl(pd.read_csv(path)) for path in sl_file_paths])
sl

cfr contacts_followed contacts_healthy  \
date       region                                                        
2014-10-31 bo                  33.7               339              336   
           bombali             24.9               944              932   
           bonthe              50.0                20               20   
           kailahun            40.8               587              587   
           kambia              28.2               375              369   
           kenema              51.7               325              321   
           koinadugu           46.7               106              102   
           kono                47.4               306              301   
           moyamba             34.4               229              229   
           port loko           13.8              5290             5247   
           pujehun             57.1               140              140   
           tonkolili           23.3              1717             1717   
           western area rural  16.1              1345              974   
           western area urban  14.5              2365             2254   
2014-12-04 bo                    30               630              630   
           bombali               24              1833             1833   
           bonthe                75               108              180   
           kailahun              41               486              486   
           kambia                32               111              110   
           kenema                53                42               42   
           koinadugu             31               247              247   
           kono                  39               546              528   
           moyamba               25               119              119   
           port loko             21              6379             6306   
           pujehun               52               119              119   
           tonkolili             23               903              903   
           western area rural    20              1962             1870   
           western area urban  18.1              5095             4896   
2014-09-07 bo                  24.6               150              146   
           bombali                9               417              404   
...                             ...               ...              ...   
2014-11-22 western area rural  16.1               NaN              NaN   
           western area urban  14.1               NaN              NaN   
2014-11-12 bo                  30.5               270              260   
           bombali             23.5              1759             1745   
           bonthe                50                 0                0   
           kailahun            40.9               517              517   
           kambia                25               379              379   
           kenema              52.3               193              193   
           koinadugu           34.5               247              247   
           kono                42.2               381              364   
           moyamba             24.8               335              332   
           port loko           14.5              6422             6358   
           pujehun             57.1               105              105   
           tonkolili           20.8              1688             1688   
           western area rural  14.5              1991             1787   
           western area urban  12.2              1574             1496   
2014-10-23 bo                    37               349              294   
           bombali               29               588              582   
           bonthe                50                20               20   
           kailahun              41               542              542   
           kambia                30               147              147   
           kenema                52 

In [51]:
def normalize_lr(raw_frame: pd.core.frame.DataFrame):
    """
    Normalize the dataframe obtained by reading a Liberia CSV to have
    one row per (region/date) pair and use columns for each variable.
    """
    
    # Normalize column names
    raw_frame.columns = map(str.lower, raw_frame.columns)
    
    DATE: str = "date"
    VARIABLE: str = "variable"
    REGION: str = "region"
    VALUE: str = "value"
    
    values_stack = []
    var_names = [] # Index of variables

    # Iterate over each variable (row)
    for index, row in raw_frame.iterrows():
        date = row[DATE]
        var_name = row[VARIABLE]
        if var_name in var_names:
            # Ignore variable: it was already defined
            continue
        var_names.append(var_name)
        
        # Iterate over each region (column)
        for col_name, value in row.iteritems():
            if col_name in {DATE, VARIABLE, "national"}:
                # Ignore: not a region
                continue
            values_stack.append({DATE: pd.to_datetime(date), REGION: col_name, VARIABLE: var_name, VALUE: value})
            
    stack_frame = pd.DataFrame(values_stack)
    pivoted = stack_frame.set_index([DATE, REGION, VARIABLE]).unstack(VARIABLE)
    # Drop to level ("value") of the column's multi index
    pivoted.columns = [col[1] for col in pivoted.columns] 
    return pivoted

lr = pd.concat([normalize_lr(pd.read_csv(path)) for path in lr_file_paths])

In [170]:
def normalize_gn(raw_frame: pd.core.frame.DataFrame):
    """
    Normalize the dataframe obtained by reading a Guinea CSV to have
    one row per (region/date) pair and use columns for each variable.
    """
    
    # Normalize column names
    raw_frame.columns = map(str.lower, raw_frame.columns)
    
    DATE: str = "date"
    VARIABLE: str = "description"
    REGION: str = "region"
    VALUE: str = "value"
    
    values_stack = []
    var_names = [] # Index of variables

    # Iterate over each variable (row)
    for index, row in raw_frame.iterrows():
        date = row[DATE]
        var_name = row[VARIABLE]
        if var_name in var_names:
            # Ignore variable: it was already defined
            continue
        var_names.append(var_name)

        # Iterate over each region (column)
        for col_name, value in row.iteritems():
            if col_name in {DATE, VARIABLE, "national", "totals"}:
                # Ignore: not a region
                continue
            values_stack.append({DATE: pd.to_datetime(date), REGION: col_name, VARIABLE: var_name, VALUE: value})
            
    stack_frame = pd.DataFrame(values_stack)
    pivoted = stack_frame.set_index([DATE, REGION, VARIABLE]).unstack(VARIABLE)
    # Drop to level ("value") of the column's multi index
    pivoted.columns = [col[1] for col in pivoted.columns] 
    return pivoted

gn = pd.concat([normalize_gn(pd.read_csv(path)) for path in gn_file_paths])

In [171]:
# Normalize the column names for shared columns
for df in [gn, sl, lr]:
    df.columns = map(str.lower, df.columns)

In [172]:
import math

def percent_to_float(str_or_nan):
    val = str_or_nan
    if type(str_or_nan) == str:
        str_or_nan = str_or_nan.replace("%", "")
        try:
            val = float(str_or_nan)
        except ValueError:
            val = float('nan')
    return val / 100

gn["fatality rate for confirmed and probables"] = gn["fatality rate for confirmed and probables"].map(percent_to_float)
sl["percent_seen"] = sl["percent_seen"].map(percent_to_float)

In [173]:
lr["$deaths"] = pd.to_numeric(lr["newly reported deaths"])
lr["$new_cases"] = pd.to_numeric(lr["new case/s (confirmed)"])
# See next cell for Guinea
# gn["$deaths"] = pd.to_numeric(gn["total deaths of confirmed"])
# gn["$new_cases"] = pd.to_numeric(gn["total cases of confirmed"])
sl["$deaths"] = pd.to_numeric(sl["death_confirmed"])
sl["$new_cases"] = pd.to_numeric(sl["new_confirmed"])

In [206]:
regional_data_frames = []

gn["total deaths of confirmed"] = pd.to_numeric(gn["total deaths of confirmed"])
gn["total cases of confirmed"] = pd.to_numeric(gn["total cases of confirmed"])
tmp_gn = gn[[*gn.columns]]
tmp_gn.reset_index(inplace=True)
tmp_gn = tmp_gn.set_index(["region", "date"])
tmp_gn.sort_index(inplace=True)


# while gnn["$deaths"].isnull().any():
#     gnn.loc[gnn["$deaths"].isnull(), "$deaths"] = gnn["$deaths"].shift()

for region in tmp_gn.index.levels[0]:
    regional_df = tmp_gn.loc[region, :].copy()
    # Compute $deaths and $new_cases as the rate of change of "total deaths of confirmed" and "total cases of confirmed"
    # After the diff, replace the first NaN by the original value (assume that the total is zero at `time == -Inf`)
    regional_df["$deaths"] = regional_df["total deaths of confirmed"].fillna(method="pad").diff().clip(0).fillna(regional_df["total deaths of confirmed"])
    regional_df["$new_cases"] = regional_df["total cases of confirmed"].fillna(method="pad").diff().clip(0).fillna(regional_df["total cases of confirmed"])
    regional_df["region"] = region
    regional_data_frames.append(regional_df)

regional_data_frames[0]

gn = pd.concat(regional_data_frames)
gn.reset_index(inplace=True)
#     gn.drop("index")
gn = gn.set_index(["date", "region"])
gn.sort_index(inplace=True)

In [ ]:
# Emit merged data frames to the disk (to better view them)

COUNTRY = "country"
REGION: str = "region"
DATE: str = "date"

# gn.rename(columns={'cumulative cfr': 'cfr'}, inplace=True)

gn.to_csv("./gn.csv")
sl.to_csv("./sl.csv")
lr.to_csv("./lr.csv")

# Anotate the country
gn[COUNTRY] = "gn"
sl[COUNTRY] = "sl"
lr[COUNTRY] = "lr"

# Concat all and reindex by country/region/date
ebola = pd.concat([gn, lr, sl])
ebola.reset_index(inplace=True)
ebola = ebola.set_index([COUNTRY, DATE, REGION])
ebola.to_csv("./ebola.csv")

### Daily average per month of new cases and deaths

In [201]:
def get_monthly_new_cases_and_deaths(df):
    # Keep only the 2 columns we need
    data = df.reset_index(level=1)[["$deaths", "$new_cases"]]
    # Sum over all the regions for each dat
    deaths = data.dropna().groupby("date").sum()
    return deaths.groupby([deaths.index.year, deaths.index.month]).mean()

## Guinea

In [202]:
get_monthly_new_cases_and_deaths(gn)

$deaths  $new_cases
date date                     
2014 8     68.6000       114.6
     9     12.9375        25.5
     10    28.0000        25.0

## Liberia

In [203]:
get_monthly_new_cases_and_deaths(lr)

$deaths  $new_cases
date date                       
2014 6      1.857143    2.142857
     7      4.272727    1.818182
     8      8.666667    8.166667
     9     37.608696    6.217391
     10    27.285714    1.476190
     11    11.500000   10.500000

## Sierra Leone

In [205]:
get_monthly_new_cases_and_deaths(sl)

$deaths  $new_cases
date date                         
2014 8      331.150000   19.600000
     9      470.642857   36.714286
     10     944.888889   58.851852
     11    1356.428571   71.785714
     12    1622.000000   54.333333